QC Utility Dashboard to Database Code

In [33]:
import os
import pyodbc
import numpy as np
import pandas as pd
from io import BytesIO
from fuzzywuzzy import process
import tableauserverclient as TSC

In [34]:
def tableau_authentication(user_name, password, site_id, tableau_cloud_url):
    tableau_auth = TSC.TableauAuth(user_name, password, site_id)  # Tableau Authentication details 'username', 'password' and 'site-id'
    server = TSC.Server(tableau_cloud_url, use_server_version=True)  # Tableau server url
    return tableau_auth, server  # Returning the variables

In [35]:

def sql_data_download(server, database, username, password, query, output_file_path):
    try:
        connection_string = ( 
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={server};"
            f"DATABASE={database};"
            f"UID={username};"
            f"PWD={password};"
        )
        connection = pyodbc.connect(connection_string)  # Connecting to the database
        df = pd.read_sql(query, connection)  # Reading the SQL database
        chart_name = query.split('GROUP')[1].replace('Y', 'y')  # Extracting the chart name from the query
        kpi_name = query.split(' ')[2].replace('SUM(', '').replace(')', '').strip()  # Extracting the KPI name from the query
        name = (kpi_name + chart_name).replace(';', '')  # Namming the file to be saved
        file_name = os.path.join(output_file_path, f"{name}.xlsx")  # Saved the file path
        df.to_excel(file_name, index=False)  # Saving the file
        connection.close()  # Closing the connection with the SQL database
    except pyodbc.Error as e:
        print(f"Failed to connect or execute the query: {e}")  # Throws a error message if the connection is not successful
    except Exception as ex:
        print(f"An error occurred: {ex}")
    print(f"Query results saved for {name}")

In [36]:
def download_parameter_data(workbook_name, parameter_name, parameter_value, save_location, tableau_auth, server):
    with server.auth.sign_in(tableau_auth):  # Server login
        all_workbooks, pagination_item = server.workbooks.get()  # Finds all the workbook present on the server
        target_workbook = next((wb for wb in all_workbooks if wb.name == workbook_name), None)  # Matches with the target workbook
        if not target_workbook:
            raise ValueError(f"Workbook {workbook_name} not found.")
        
        server.workbooks.populate_views(target_workbook)  # Finds all the sheets present in the dashboard 
        for target_view in target_workbook.views:  # The loop goes through all the sheets present in the workbook one by one
            sheet_name = target_view.name
            options = TSC.CSVRequestOptions()
            options.vf(parameter_name, parameter_value)  # Parameter is applied
            try:
                server.views.populate_csv(target_view, req_options=options)  # The sheet data is extracted
            except TSC.ServerResponseError as e:
                print(f"Error querying data for Parameter {parameter_value}: {e}")  # Error message if server does not respond
                continue

            csv_data = BytesIO(b''.join(target_view.csv))  # The sheet data is converted to csv format
            df = pd.read_csv(csv_data)  # Read into pandas DataFrame
            df = df.drop_duplicates()  # Remove duplicates to ensure clean data
            file_name = os.path.join(save_location, f"{parameter_value} {sheet_name}.xlsx")  # Save each parameter's data into a separate Excel file
            df.to_excel(file_name, index=False)  # Saves the file
    print(f"{parameter_value} data downloaded for {workbook_name} to location! ")

In [37]:
def save_tableau_data(control_df, save_location, tableau_auth, server):
    for f in range(len(control_df)):
        workbook_name  = control_df.loc[f,'Dashboard']  # Targetted dashboard name
        parameter_name = control_df.loc[f,'Parameter Name']  # Parameter in the workbook
        parameter_value = control_df.loc[f,'Parameter Value']  # All the values of the parameter
        if workbook_name is np.nan:  # Breaks the loop if the cell is empty in the Control_File.xlsx
            break
        else:
            download_parameter_data(workbook_name, parameter_name, parameter_value, save_location, tableau_auth, server)  # Downloading the data

In [38]:
def save_sql_data(control_df, server_address, database, username, password, save_location):
    for f in range(len(control_df)):
        query = control_df.loc[f, 'QC_Query']  # Passing the queries one by one to the function
        
        sql_data_download(server_address, database, username, password, query, save_location)  # Downloades the data according to the query passed

In [39]:
def compare_data_by_measure(source_file_path, qc_file_path, file, compare_measure, column_name, output_path):
    sdf = pd.read_excel(source_file_path)  # Reads the source data file
    qcdf = pd.read_excel(qc_file_path)  # Reads the database data file
    sdf = sdf.rename(columns={process.extractOne(compare_measure, sdf.columns)[0]: compare_measure,})  # Changes the column name to generalize
    qcdf = qcdf.rename(columns={process.extractOne(compare_measure, qcdf.columns)[0]: compare_measure})  # Changes the column name to generalize
    for col, df, file_name in [(compare_measure, sdf, file),  # Selecting the columns to be compared with respect to the measure they are categorized
                               ("Measure Calculation", sdf, file), 
                               (compare_measure, qcdf, file), 
                               (column_name, qcdf, file)]:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in {file_name}.")
    merged_df = pd.merge(sdf[[compare_measure, "Measure Calculation"]],  # Merging the data which does not match
                         qcdf[[compare_measure, column_name]], 
                         on=compare_measure, 
                         how='outer',  
                         suffixes=('_file1', '_file2'))
    mismatched_rows = merged_df[merged_df["Measure Calculation"] != merged_df[column_name]] 
    mismatched_rows.to_excel(output_path, index=False)  # Saving the file

In [40]:
def compare_data(source_workbook_data_location, database_data_location, compared_data_location):
    source_files = {f for f in os.listdir(source_workbook_data_location) if os.path.isfile(os.path.join(source_workbook_data_location, f))}  # Reading all the files in the source workbook data folder
    qc_files = {f for f in os.listdir(database_data_location) if os.path.isfile(os.path.join(database_data_location, f))}  # Reading all the files in the source database data folder
    for qc_file in qc_files:  
        file_name = process.extractOne(qc_file, source_files)[0]  # Matching with the same file using the name of the file
        source_file_path = os.path.join(source_workbook_data_location, file_name)  # Creating the path to read files
        qc_file_path = os.path.join(database_data_location, qc_file)
        compared_file_path = os.path.join(compared_data_location, f'{file_name}')
        compare_measure = file_name.split(' ')[2].replace('.xlsx', '')  # Extracting the measure name for comparison
        kpi_name = file_name.split(' ')[0]  # Extracting the KPI name to match with the column name
        df = pd.read_excel(qc_file_path)
        column_name = process.extractOne(kpi_name, df.columns)[0]  # Matching the KPI name with the column name in the database file
        try:
            compare_data_by_measure(source_file_path, qc_file_path, file_name, compare_measure, column_name, compared_file_path)  # Comparing and saving the results
        except Exception as e:
                print(f"Error comparing '{file_name}': {e}")
    print(f"Comparison result are saved!!")
            

In [41]:
def output_feed(compared_data_location, output_file_location, database):
    files = {f for f in os.listdir(compared_data_location) if os.path.isfile(os.path.join(compared_data_location, f))}  # Reading all the comparison results
    diff_data = []  # Initialized an empty list to append the data
    for file in files:
        source_db = []  # Initialized an empty list to append the source workbook data points
        qc_db = []  # Initialized an empty list to append the database data points
        measure = []  # Initialized an empty list to append the measure name
        file_path = os.path.join(compared_data_location, file)  # Created the file path
        chart_name = file.replace('.xlsx', '').split(' ')[2]  # Extracting the chart name
        measure_name = file.split(' ')[2].replace('.xlsx', '')  # Extracting the measure name
        kpi_name = file.split(' ')[0]  # Extracting the KPI name
        df = pd.read_excel(file_path)
        column_name = process.extractOne(kpi_name, df.columns)[0]  # Matching with the column to read
        sdb = df['Measure Calculation'].dropna()  # Extracting the values
        qcdb = df[column_name].dropna()
        measure_db = df[measure_name].dropna()  # Dropping the Null Values
        for n in sdb:
            source_db.append(float(n.replace(',', '')))  # Storing the one by one into a list by converting the data type to float
        for n in qcdb:
            qc_db.append(float(n))  # Storing the one by one into a list by converting the data type to float
        for n in measure_db:
            measure.append(n)
        for i in range(len(source_db)):  # Storing all the data in a list which can then be saved in an excel file
            diff_data.append({
                'Database Name': database,
                'KPI': kpi_name,
                'Categorized BY': chart_name,
                'Measure': measure[i],
                'Source Workbook Data Points': source_db[i],
                'Database Data Points': qc_db[i]
            })
    odf = pd.DataFrame(diff_data)  # Storing the data into a data frame
    output_file = os.path.join(output_file_location, f"Output Feed.xlsx")
    odf.to_excel(output_file, index=False)  # Saving output
    print("Results are Saved!!")
            

In [42]:
# Folder location to save Source Workbook, QC Workbook, and Compared data, Output Feed File, Control File and Tableau Auth Details File
source_workbook_data_location = "/workspaces/Tableau_Python/Workbook Data/Dashboard to Database/Source Workbook Data"
sql_server_authentication_details_file_location = "/workspaces/Tableau_Python/SQL Server Auth Details.txt"
database_data_location = "Workbook Data/Dashboard to Database/Database Data"
compared_data_location = "Workbook Data/Dashboard to Database/Compared Data"
tableau_authentication_details_file_location = "Tableau Auth Details.txt"
output_file_location = "Workbook Data/Dashboard to Database"
control_file_location = "/workspaces/Tableau_Python/Control_File.xlsx"

In [43]:
# Reading the Control File: SQL Server Auth Details
with open(sql_server_authentication_details_file_location, 'r') as details:
    user_details = details.readlines()
server_address = user_details[0].strip().replace("Server Address: ", "")  # SQL server address
database = user_details[1].strip().replace("Database Name: ", "")  # Database Name
username_sql = user_details[2].strip().replace("User Name: ", "")  # Username to login in to the server
password_sql = user_details[3].strip().replace("Password: ", "")  # Password

In [44]:
# Reading the Control File: Tableau Auth Details
with open(tableau_authentication_details_file_location, 'r') as details:
    user_details = details.readlines()
user_name = user_details[0].strip().replace('User Name: ', '')  # Tableau Cloud User Name
password = user_details[1].strip().replace('Password: ', '')  # Tableau Cloud Password
site_id = user_details[2].strip().replace('Site-ID: ', '')  # Tableau Cloud Site-ID
tableau_cloud_url = user_details[3].strip().replace('Cloud URL: ', '')  # Tableau Cloud URL

In [45]:
# Reading the Control_File.xlsx
control_df = pd.read_excel(control_file_location)
control_df['row number'] = control_df.reset_index().index  # Reset the index
control_source_df = control_df[['row number', 'Dashboard', 'Parameter Name', 'Parameter Value']]  # Reading the columns required into a dataframe
control_db_df = control_df[['QC_Query']]

In [46]:
tableau_auth, server = tableau_authentication(user_name, password, site_id, tableau_cloud_url)  # Authenticating with the server

In [47]:
# save_tableau_data(control_source_df, source_workbook_data_location, tableau_auth, server)  # Downloading and Saving the data from the Tableau Dashboard
save_sql_data(control_db_df, server_address, database, username_sql, password_sql , database_data_location)  # Downloading and Saving the data from SQL database

Failed to connect or execute the query: ('HYT00', '[HYT00] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')


UnboundLocalError: cannot access local variable 'name' where it is not associated with a value

In [ ]:
compare_data(source_workbook_data_location, database_data_location, compared_data_location)  # Comparing data

Comparison result are saved!!


In [ ]:
output_feed(compared_data_location, output_file_location, database)  # Saving the Output Feed File for Tableau QC Dashboard

Results are Saved!!
